In [1]:
############MM
## imports
##############
import sys
sys.path.append ('..\TDAAPI')
import a_TDA_IF 
import a_utils
import talib
import pandas as pd
import numpy as np
import datetime

In [3]:
###################################
class Stock:
    
    def __init__(self, Symbol, Company=''):
        self.CompanyNm = Company
        self.Status= 0
        self.Symbol= Symbol
        self.Sector=''
        self.Industry=''
        self.StockBeta=1

        self.SMADays=10
        self.SMAAlert=0
        self.SMAState=0 # above or below
        self.SMA=0
        self.SMADate=None  #datetime obj
        
        self.Broker=''
        self.Shares=0
        self.HistDF=None   #dataframe
        self.HistStartDate=0 # Epoc second
        self.Price=0
        self.Volume=0
        self.HistEndDate=0 # Epoc second
        self.PriceDate=None  # datetime obj

        self.Comment='' # for collecting all changes, warnings
    

    def GetHist(self):
    # get market data
        print("get symbol market data, ok")

        #print ('end date', yEDate)
        #print ('start Date', ySDate)

        ###################self.HistDF=a_TDA_IF.TDA_Price_Hist (Symbol=self.Symbol, StartDTStamp=self.HistStartDate, EndDTStamp=self.HistEndDate )
        yDF2=pd.read_csv(r'C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug\GOOG_TDA_raw.csv')
        self.HistDF= yDF2.iloc[:, 1:]

        
        self.Comment=self.Comment + "got quote update, " + str(type(self.HistDF)) + str(self.HistDF.shape) +'; ' 

        self.UpdateTA()

        self.SaveHist()

    def UpdateTA(self):
        print('update stock technical indicators')
        
        #

        SMAPeriod=self.SMADays
        if SMAPeriod > 100 or SMAPeriod < 1 :
            SMAPeriod =10 
            self.Comment=self.Comment + "SMADays is out of range of 1 to 100, reset to 10; " 

        #https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/
        ySMA=talib.SMA(self.HistDF['close'].values, timeperiod=SMAPeriod)

        # df1["SMA"]=ySMA ##works   
        self.HistDF.insert(5,"SMA",  ySMA, True) #works too 
        
        # other new value for OLI fields
        # find the lastest data

        self.Price=self.HistDF.iloc[-1, 4]
        self.Volume=self.HistDF.iloc[-1,6]
        self.PriceDate= a_utils.TDAepoch2DT(self.HistDF.iloc[-1,7] )

        self.Comment=self.Comment + "updated price=" + str(self.Price) +' volume=' +str(self.Volume) + ' date=' + str(self.PriceDate) +'; '

        self.SMA=self.HistDF.iloc[-1,5]  # column 5 is sma 
        if self.SMAState==1 and self.SMA >= self.Price:
            self.SMAAlert = -1
            self.SMAState =0  
            self.SMADate=self.PriceDate
            self.Comment=self.Comment + "SMA Alert: price dropped below SMA" +'; '
        elif self.SMAState==0 and self.SMA < self.Price:
            self.SMAAlert = 1
            self.SMAState =1 
            self.SMADate=self.PriceDate
            self.Comment=self.Comment + "SMA Alert: price rose above SMA"+'; '
        else:
            self.SMAAlert = 0 
            self.Comment=self.Comment + "SMA Alert reset since no change since last update" +'; ' 

    def SaveHist(self):
        print('save stock data to file')

        cdir=self.Symbol
        pdir=r'C:\Users\bt\Documents\GitHub\SigmaCodingBTC\TDAAPI\historical_data\a_Debug'
        path= a_utils.addDir(pdir, cdir)

        path=a_utils.DF2CSV(self.HistDF, path, self.Symbol, '')

        self.Comment=self.Comment + "; Update saved to " + str(path) +'; '



    

In [1]:
#testng class Stock
yStock=Stock(Symbol='AMD')

yStock.HistEndDate=a_utils.epoch_date_stamp()
yStock.HistStartDate=a_utils.epoch_from_today( Yr=0, Mo=0, Day=15) 

yStock.GetHist()

print (yStock.HistDF)
print (yStock.HistDF.iloc[-1,7])
print (yStock.Price)
print   (yStock.Volume)
print    (yStock.PriceDate) 
print   (type(yStock.PriceDate))
print (yStock.Comment)

NameError: name 'Stock' is not defined

In [ ]:
df7 = pd.DataFrame(
{"a" : [4 ,5, 6], 
"b" : [7, 8, 9], 
"c" : [10, 11, 12]},    
index = [1, 2, 3])

print (df7.shape, type(df7))
print (df7.iloc[-1, 0])

(3, 3) <class 'pandas.core.frame.DataFrame'>
6
